# Hardware test

In [1]:
# for performance monitoring
import time
# for hardware info
import torch
import platform
import psutil

Stampa la versione di PyTorch e seleziona automaticamente il device disponibile (`cuda`, `mps` o `cpu`). Se PyTorch non è installato, viene gestita l'eccezione e viene mostrato un messaggio informativo.

In [ ]:
try:
    print(f"PyTorch version: {torch.__version__}\n")

    if torch.cuda.is_available(): # only for Nvidia GPUs
        device = torch.device("cuda")
        device_name = torch.cuda.get_device_name(0)
    elif torch.backends.mps.is_available(): # only for Mac
        device = torch.device("mps")
        device_name = "Apple GPU (MPS)"
    else:
        device = torch.device("cpu")
        device_name = platform.processor() or "CPU"

except ImportError:
    print("PyTorch non è installato")
    

PyTorch version: 2.9.1



Dettagli hardware: vengono mostrati i dati della GPU (se `cuda` è attivo) o una nota su MPS; in alternativa vengono riportate le informazioni sulla CPU (core, thread, frequenza).

In [15]:
if device.type == "cuda":
    # Informazioni sulle GPU
    num_gpus = torch.cuda.device_count()
    print(f"Numero di GPU disponibili: {num_gpus}\n")

    for i in range(num_gpus):
        prop = torch.cuda.get_device_properties(i)
        print(f"GPU {i}: {prop.name}")
        print(f"  Memoria totale: {prop.total_memory / (1024 ** 3):.2f} GB")
        print(f"  Multiprocessori: {prop.multi_processor_count}")
        print(f"  Capabilities: {prop.major}.{prop.minor}\n")

elif device.type == "mps":
    print("== Informazioni GPU (MPS / Apple Silicon) ==")
    print("MPS è disponibile e attivo tramite Metal Performance Shaders.\n")

else:
    # Informazioni sulla CPU
    print("== Informazioni CPU ==")
    print(f"Nome CPU: {platform.processor() or 'N/D'}")
    print(f"Numero di core fisici: {psutil.cpu_count(logical=False)}")
    print(f"Numero di thread logici: {psutil.cpu_count(logical=True)}")
    print(f"Frequenza CPU: {psutil.cpu_freq().current / 1000:.2f} GHz\n")


== Informazioni GPU (MPS / Apple Silicon) ==
MPS è disponibile e attivo tramite Metal Performance Shaders.



Test di prestazioni: esegue moltiplicazioni matriciali casuali sul device selezionato, misura i tempi per ogni operazione e calcola un throughput medio espresso in TFLOPS.

In [ ]:
import torch
import time

# Funzione helper per sincronizzare in base al device
def sync_device(dev):
    if dev.type == "cuda":
        torch.cuda.synchronize()
    elif dev.type == "mps":
        torch.mps.synchronize()
    # La CPU è sincrona, non serve fare nulla

# compute m multiplications between (size x size) matrices
size = 5000
m = 50
x = torch.rand(size, size, device=device)
y = torch.rand(size, size, device=device)

flops_per_op = 2 * size**3 - size**2

# ---------------------------------------------------------
# A. WARM-UP (Riscaldamento)
# Serve a inizializzare le cache e compilare i kernel
# ---------------------------------------------------------
print("Eseguendo warm-up...")
for _ in range(5):
    _ = x @ y
sync_device(device) # Assicura che il warm-up sia finito davvero

# ---------------------------------------------------------
# B. BENCHMARK
# ---------------------------------------------------------
print(f"Inizio benchmark ({m} iterazioni)...")

# Usiamo perf_counter per alta precisione
start_time = time.perf_counter()

for i in range(m):
    z = x @ y

sync_device(device)
end_time = time.perf_counter()

total_time = end_time - start_time
avg_time_per_op = total_time / m

# Totale FLOPs calcolati nel loop
total_flops = flops_per_op * m
tflops = (total_flops / total_time) / 1e12

print(f"Tempo totale per {m} moltiplicazioni tra matrici di {size}x{size} elementi: {total_time:.4f} secondi")
print(f"FLOPS medi: {tflops:.2f} TFLOPS")

== Test di moltiplicazione matriciale ==
Tempo totale per 10 moltiplicazioni tra matrici di 5000x5000 elementi: 0.3747 secondi
FLOPS medi: 6.82 TFLOPS
